In [2]:
import cv2

In [3]:
# distance from camera to object(face) measured 
# centimeter 
Known_distance = 76.2
  
# width of face in the real world or Object Plane 
# centimeter 
Known_width = 14.3
  
# Colors 
GREEN = (0, 255, 0) 
RED = (0, 0, 255) 
WHITE = (255, 255, 255) 
BLACK = (0, 0, 0) 
  
# defining the fonts 
fonts = cv2.FONT_HERSHEY_COMPLEX 
  
# face detector object 
face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [4]:
# focal length finder function 
def Focal_Length_Finder(measured_distance, real_width, width_in_rf_image): 
  
    # finding the focal length 
    focal_length = (width_in_rf_image * measured_distance) / real_width 
    return focal_length 

In [5]:
# distance estimation function 
def Distance_finder(Focal_Length, real_face_width, face_width_in_frame): 
  
    distance = (real_face_width * Focal_Length)/face_width_in_frame 
  
    # return the distance 
    return distance 

In [9]:
def face_data(image): 
  
    face_width = 0  # making face width to zero 
  
    # converting color image to gray scale image 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # detecting face in the image 
    faces = face_detector.detectMultiScale(gray_image, 1.3, 5) 
  
    # looping through the faces detect in the image 
    # getting coordinates x, y , width and height 
    for (x, y, h, w) in faces: 
  
        # draw the rectangle on the face 
        cv2.rectangle(image, (x, y), (x+w, y+h), GREEN, 2) 
  
        # getting face width in the pixels 
        face_width = w 
  
    # return the face width in pixel 
    return face_width 
  
  
# reading reference_image from directory 
ref_image = cv2.imread(r"D:\ANACONDA3\envs\cam_dist\refs\1st.jpg") 
  
# find the face width(pixels) in the reference_image 
ref_image_face_width = face_data(ref_image) 
  
# get the focal by calling "Focal_Length_Finder" 
# face width in reference(pixels), 
# Known_distance(centimeters), 
# known_width(centimeters) 
Focal_length_found = Focal_Length_Finder( 
    Known_distance, Known_width, ref_image_face_width) 
  
print(Focal_length_found) 
  
# show the reference image 
cv2.imshow(r"D:\ANACONDA3\envs\cam_dist\refs\1st.jpg", ref_image) 
  
# initialize the camera object so that we 
# can get frame from it 
cap = cv2.VideoCapture(0) 
  
# looping through frame, incoming from  
# camera/video 
while True: 
  
    # reading the frame from camera 
    _, frame = cap.read() 
  
    # calling face_data function to find 
    # the width of face(pixels) in the frame 
    face_width_in_frame = face_data(frame) 
  
    # check if the face is zero then not  
    # find the distance 
    if face_width_in_frame != 0: 
        
        # finding the distance by calling function  
        # Distance finder function need  
        # these arguments the Focal_Length, 
        # Known_width(centimeters), 
        # and Known_distance(centimeters) 
        Distance = Distance_finder(Focal_length_found, Known_width, face_width_in_frame) 
  
        # draw line as background of text 
        cv2.line(frame, (30, 30), (230, 30), RED, 32) 
        cv2.line(frame, (30, 30), (230, 30), BLACK, 28) 
  
        # Drawing Text on the screen 
        cv2.putText( 
            frame, f"Distance: {round(Distance,2)} CM", (30, 35),fonts, 0.6, GREEN, 2) 
  
    # show the frame on the screen 
    cv2.imshow("frame", frame) 
  
    # quit the program if you press 'q' on keyboard 
    if cv2.waitKey(1) == ord("q"): 
        break
  
# closing the camera 
cap.release() 
  
# closing the windows that are opened 
cv2.destroyAllWindows() 

1513.3426573426573


In [4]:
import cv2
import numpy as np

# Load COCO class labels
classes = open('coco.names').read().strip().split('\n')

# Load pre-trained MobileNet SSD model
net = cv2.dnn.readNetFromTensorflow(r"D:\ANACONDA3\envs\cam_dist\measure_distance_using_camera\frozen_inference_graph.pb", r"D:\ANACONDA3\envs\cam_dist\measure_distance_using_camera\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt")

# Open a video capture object (use 0 for default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Prepare the frame for object detection
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

    # Set the input to the network and perform forward pass
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections
        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])
            label = classes[class_id]
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, label, (startX, startY - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the output
    cv2.imshow('Object Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range